# Toothbrush Sales - Data Analysis

In [4]:
import pandas as pd
import numpy as np
import re
import geopandas
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
from datetime import datetime
from matplotlib import font_manager
font_manager.findSystemFonts(fontpaths=None, fontext="ttf")
from matplotlib.colors import LinearSegmentedColormap
from configparser import ConfigParser
import pymysql

In [7]:
# Load clean Orders data from RDS

# Set up config parser
parser = ConfigParser()
parser.read("ec2_config.ini")

# Set ec2 params
ec2_params = parser["ec2"]

# Connect to rds
db_params = parser["mysql"]

db_connector = pymysql.connect(
    host=db_params["host"],
    user=db_params["user"],
    password=db_params["password"],
    database=db_params["database"],
    autocommit=True,
)

# Get Orders data from RDS
cursor = db_connector.cursor()
cursor.execute("SELECT * FROM Orders;")
columns = [
    "Order Number",
    "Toothbrush Type",
    "Order Date",
    "Customer Age",
    "Order Quantity",
    "Delivery Postcode",
    "Billing Postcode",
    "is_first",
    "Dispatch Status",
    "Dispatched Date",
    "Delivery Status",
    "Delivery Date"
]
Orders = pd.DataFrame(cursor.fetchall(), columns=columns)

In [ ]:
# Create area postcode for delivery and billing postcodes
Orders["Delivery Area"] = Orders["Delivery Postcode"].str.extract(r"(^[A-Z]+)")
Orders["Billing Area"] = Orders["Billing Postcode"].str.extract(r"(^[A-Z]+)")

In [ ]:
# Set matplotlib style 
font = {'family': 'HelveticaNowText-Light',
        'size': 20}

plt.rcParams['text.color'] = "#1D1D40"
plt.rc('font', **font)

## Delivery Status by Hour of the Day

In [ ]:
# Plot unsuccessful deliveries and dispatch hour
df = Orders[["Order Number", "Dispatched Date", "Delivery Status"]].copy()

# Add hour order was placed at
df["Dispatched Hour"] = df["Dispatched Date"].dt.hour

# Group df
x = pd.DataFrame(df.groupby([df["Dispatched Hour"], df["Delivery Status"]])["Order Number"].count().reset_index())
x.rename({"Order Number": "Number of Orders"}, axis=1, inplace=True)

# Set varaibles
delivery_status = ['In Transit', 'Delivered', 'Unsuccessful', np.NaN]

# Set figure
fig, ax = plt.subplots(figsize=(20,10))

# Plot data
plt.plot(x["Dispatched Hour"][x["Delivery Status"]==delivery_status[0]], x["Number of Orders"][x["Delivery Status"]==delivery_status[0]], label=delivery_status[0], linewidth=3, color="#FCBA55")
plt.plot(x["Dispatched Hour"][x["Delivery Status"]==delivery_status[1]], x["Number of Orders"][x["Delivery Status"]==delivery_status[1]], label=delivery_status[1], linewidth=3, color="#32ACFF")
plt.plot(x["Dispatched Hour"][x["Delivery Status"]==delivery_status[2]], x["Number of Orders"][x["Delivery Status"]==delivery_status[2]], label=delivery_status[2], linewidth=3, color="#F16B83")

# Set title
plt.title("Delivery Status by Hour of the Day", y=1.05, fontweight="bold")

# Plot x-axis
plt.xlabel("Hour of Dispatch (24-Hours)", fontweight="bold", labelpad=15)
plt.xticks(range(0,24,1))
plt.xlim(xmin=0, xmax=24)

# Set legend
plt.legend(loc="upper right", frameon=False, title="Key")

# Set y-axis
plt.ylabel("Number of Orders", fontweight="bold")
plt.ylim(ymin=0, ymax=500)
plt.yticks(range(0,500,50))

plt.show()

In [ ]:
# Plot order status before and after 4 A.M.
df = Orders[["Order Number", "Dispatched Date", "Delivery Status"]].copy()

# Add hour order was placed at
df.loc[df["Dispatched Date"].dt.hour > 4, "After 4"] = "True"
df.loc[df["Dispatched Date"].dt.hour < 4, "After 4"] = "False"

# Remove nans and dispatch date col
df.dropna(how="any", inplace=True)
df.drop("Dispatched Date", inplace=True, axis=1)

# Get totals of before and after 4
before_4 = df[df["After 4"]=="False"]["Order Number"].count()
after_4 = df[df["After 4"]=="True"]["Order Number"].count()

# Group df
x = pd.DataFrame(df.groupby([df["After 4"], df["Delivery Status"]])["Order Number"].count().reset_index())
x.rename({"Order Number": "Number of Orders"}, axis=1, inplace=True)

x = x.pivot("After 4", "Delivery Status", "Number of Orders")

# Replace nans with 0
x.replace({np.nan: 0}, inplace=True)

# Calculate proportions of orders
x.loc["False"] = x.loc["False"].apply(lambda a: round((a/before_4)*100))
x.loc["True"] = x.loc["True"].apply(lambda a: round((a/after_4)*100))

# Rename col
x.rename({"Number of Orders": "Proportion of Orders"}, axis=1, inplace=True)

# Set varaibles
delivery_status = ['In Transit', 'Delivered', 'Unsuccessful']


# Plot data
ax = x.plot(kind="bar", figsize=(8,10), color={'In Transit': "#FCBA55", 'Delivered': "#32ACFF", 'Unsuccessful': "#F16B83"}, width=0.8)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Set title
plt.title("Delivery Statuses Before and After 4 A.M.", y=1.05, fontweight="bold")

# Set x-axis
plt.xlabel(None)
plt.xticks(ticks=np.arange(2) ,rotation="horizontal", labels=["Before 4 A.M.", "After 4 A.M."])

# Set legend
plt.legend(loc="upper right", frameon=False, title="Key", bbox_to_anchor=(1.2,1))


# Set y-axis
plt.ylabel("Proportion of Orders (%)", fontweight="bold")
plt.ylim(ymin=0)
plt.yticks(range(0,100,10))

plt.show()

## Customer Age and Toothbrush Type Analysis

In [ ]:
# Plot number of orders by customer age and toothbruh type
df = Orders[["Customer Age", "Toothbrush Type", "Order Number"]]

# Set variables
bins = list(range(0, 120, 5))
x1 = df["Customer Age"][df["Toothbrush Type"]=="Toothbrush 2000"]
x2 = df["Customer Age"][df["Toothbrush Type"]=="Toothbrush 4000"]

# Set up plot
fig, ax = plt.subplots(figsize=(20,10))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Plot data
plt.hist(x1, bins=bins, label="Toothbrush 2000", color="#06A099", alpha=0.7)
plt.hist(x2, bins=bins, label="Toothbrush 4000", color="#FCBA55", alpha=0.7)

# Set title
plt.title("Number of Toothbrush Orders by Customer Age and Toothbrush Type", y=1.05, fontweight="bold")

# Set x-axis
plt.xlabel("Customer Age (Years)", labelpad=15, fontweight="bold")
plt.xticks(list(range(0, 120, 5)))
plt.xlim(xmin=0, xmax=120)

# Set y-axis
plt.ylabel("Number of Orders", fontweight="bold")
plt.ylim(ymin=0, ymax=1200) 
plt.legend(loc="upper right",frameon=False, title="Key")

# Display plot
plt.show()

## Number of Orders by Toothbrush Type during the Day

In [ ]:
# Plot number of orders by time of day and toothbrush type
df = Orders[["Order Number", "Toothbrush Type", "Order Date"]].copy()

# Add hour order was placed at
df["Order Hour"] = df["Order Date"].dt.hour

x = pd.DataFrame(df.groupby([df["Order Hour"], df["Toothbrush Type"]])["Order Number"].count().reset_index())
x.rename({"Order Number": "Number of Orders"}, axis=1, inplace=True)

# Set figure
fig, ax = plt.subplots(figsize=(20,10))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Plot data
plt.plot(x["Order Hour"][x["Toothbrush Type"]=="Toothbrush 2000"], x["Number of Orders"][x["Toothbrush Type"]=="Toothbrush 2000"], label="Toothbrush 2000", color="#06A099", linewidth=3)
plt.plot(x["Order Hour"][x["Toothbrush Type"]=="Toothbrush 4000"], x["Number of Orders"][x["Toothbrush Type"]=="Toothbrush 4000"], label="Toothbrush 4000", color="#FCBA55", linewidth=3)
plt.title("Number of Toothbrush Orders by Order Hour and Toothbrush Type", fontweight="bold", y=1.05)

# Set x-axis
plt.xlabel("Hour of Order (24-Hours)", fontweight="bold", labelpad=15)
plt.xticks(range(0,25,1))
plt.xlim(xmin=0)

# Set y-axis
plt.ylabel("Number of Orders", fontweight="bold")
plt.ylim(ymin=150, ymax=250)
plt.yticks(range(0,500,50))


plt.legend(loc="upper right",frameon=False, title="Key")
plt.show()

## Postcode Analysis

### Postcode Database Data

In [ ]:
# Load postcode_db
col_names=[
    "Postcode",
    "Live",
    "User",
    "Country"
]
postcode_db = pd.read_csv("./data/open_postcode_geo.csv", usecols=[0,1,2,6], names=col_names, na_values="\\N")

# Keep only live postcodes and drop column
postcode_db = postcode_db[postcode_db["Live"]=="live"]
postcode_db.drop("Live", axis=1, inplace=True)

# Add postcode area column
postcode_db["Postcode Area"] = postcode_db["Postcode"].str.extract(r"(^[A-Z]+)")

# Drop GIR from postcode_db as not a geographic postcode
postcode_db = postcode_db[postcode_db["Postcode Area"]!="GIR"]

# Write postcode_db to csv
postcode_db.to_csv("../postcode_db.csv")

### UK and Channel Islands Map Data

In [ ]:
# Load uk_polygon
uk_polygon = geopandas.read_file("./polygons/polygon_package/postcode_polygons.gpkg")

# Drop cat col
uk_polygon.drop("cat", axis=1, inplace=True)

# Drop pc areas not in postcode_db
x = postcode_db["Postcode Area"].unique()
y = uk_polygon["pc_area"].unique()
unknown_pc = list(set(y).difference(set(x))) #serveral unregistered postcode areas - seems to be London subdistricts

# Drop unknown pcs
uk_polygon = uk_polygon[~(uk_polygon["pc_area"].isin(unknown_pc))]

# Load channel islands (IM, GY, JE) polygons
guernsey = geopandas.read_file("./polygons/Guernsey/POLYGON.shp")
iom = geopandas.read_file(r"./polygons/Isle of Man/POLYGON.shp")
jersey = geopandas.read_file("./polygons/Jersey/POLYGON.shp")

# Add polygons of channel islands to uk_polygons
channel_islands = pd.DataFrame({
    "pc_area": ["GY", "IM", "JE"],
    "geometry": [guernsey.iloc[0,0], iom.iloc[0,0], jersey.iloc[0,0]] 
})

uki = pd.concat([uk_polygon, channel_islands], ignore_index=True)

# Change column names
uki.columns = ["Postcode Area", "geometry"]

# Dissolve uki by Postcode Area
uki = uki.dissolve(by="Postcode Area")

### Population Data

In [ ]:
# Get population of Eng and Wales by pa based on 2011 census
population_eng_wales = pd.read_csv("./geo_data/population_Eng_Wales.csv", usecols=["geography", "Variable: All usual residents; measures: Value"])

# Extract pa and name from geography column
population_eng_wales["Postcode Area"] = population_eng_wales["geography"].str.split(" - ").str[0]
population_eng_wales["Postcode Area Name"] = population_eng_wales["geography"].str.split(" - ").str[-1]

# Drop geography col
population_eng_wales.drop("geography", axis=1, inplace=True)

# Rename cols
population_eng_wales.rename({"Variable: All usual residents; measures: Value" : "Population"}, axis=1, inplace=True)

# Remove TD row as Galashiels population will be included in Soctland population of TD postcodes
population_eng_wales = population_eng_wales[population_eng_wales["Postcode Area"]!="TD"]

# Get postcode areas without name/population
pa_names = {
       'AB': "Aberdeen" , 
       'DD': "Dundee", 
       'DG': "Dumfries", 
       'EH': "Edinburgh", 
       'FK': "Falkrik", 
       'G': "Glasgow", 
       'GY': "Guernsey", 
       'HS': "Hebrides", 
       'IM': "Isle of Man", 
       'IV': "Inverness", 
       'JE': "Jersey",
       'KA': "Kilmarnock", 
       'KW': "Kirkwall", 
       'KY': "Kirkcaldy", 
       'ML': "Motherwell", 
       'PA': "Paisley", 
       'PH': "Perth",
       'TD': "Galashiels", 
       'ZE': "Lerwick"
}

# Load Scotland postcode populations data
scotland_population = pd.read_csv("./scotland_population.csv", usecols=[0,1])

# Add postcode area column
scotland_population["Postcode Area"] = scotland_population["Postcode"].str.extract("^([A-Z]+)")
scotland_population["Postcode"].unique()

# Drop Postcode column
scotland_population.drop("Postcode", axis=1, inplace=True)

# Rename cols
scotland_population.rename({"All people": "Population"}, axis=1, inplace=True)

# Change population to int
scotland_population["Population"] = scotland_population["Population"].str.replace(",", "")
scotland_population["Population"] = scotland_population["Population"].astype('int')

# Sum population by Postcode Area
scotland_population = scotland_population.groupby(scotland_population["Postcode Area"]).sum().reset_index()
scotland_population["Postcode Area Name"] = scotland_population["Postcode Area"].replace(pa_names)

# Add IM, GY, JE populations
ci_population = pd.DataFrame({
    "Postcode Area": ["IM", "JE", "GY"],
    "Population": [84889, 97857, 62915],
    "Postcode Area Name": ["Isle of Man", "Jersey", "Guernsey"]
})

# Concat population_eng_wales and scotland_population
population = pd.concat([population_eng_wales, scotland_population, ci_population], ignore_index=True)

### Map Visualisation

In [ ]:
# Group Number of Orders by Delivery Postcode Area
df = Orders[["Order Number", "Delivery Area"]].copy()
df = pd.DataFrame(df.groupby(df["Delivery Area"])["Order Number"].count().reset_index())

# Rename col
df.rename({"Delivery Area": "Postcode Area", "Order Number": "Number of Orders"}, axis=1, inplace=True)

# Merge with population data
df = df.merge(population, on="Postcode Area", how="left")

# Merge quantity_by_pa with uki
df = uki.merge(df, how="right", on="Postcode Area")

# Add boundary column
df["Boundary"] = df.boundary

# Add order quantity per 1000
df["per 100000"] = round((df["Number of Orders"] / df["Population"]) * 100000)

In [ ]:
# Plot number of orders per 100 000 people in each area

# Set figure
fig, ax = plt.subplots(figsize=(15,20))
ax.set_aspect('equal')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

cmap = LinearSegmentedColormap.from_list("", ["#FCBA55", "#32ACFF", "#1D1D40"])

# Plot data
df.plot(ax=ax, column="per 100000", cmap=cmap, legend=True)
df["Boundary"].plot(ax=ax, color="#1D1D40", linewidth=0.5)

# Set title
plt.title("Number of Orders by Postcode Area per 100 000", fontweight="bold")

# Set x-axis
plt.xticks([])

# Set y-axis
plt.yticks([])

plt.show();